# 네이버 쇼핑리뷰 감성분석
- Mecab 형태소 분석기

### Mecab 설치

In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 35.8 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-02-11 00:43:26--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20filename%3D%22mecab-0.996-ko-0.9.2.tar.gz%22&response-co

### 1. 데이터 탐색

In [5]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df = pd.read_table(url, names=['ratings','reviews'])
df.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [7]:
# 평점이 4, 5 점은 긍정(1), 나머지는 부정(0)
df['label'] = df.ratings.apply(lambda x: 1 if x >= 4 else 0)
df.head()

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [8]:
# label 분포
df.label.value_counts()

0    100037
1     99963
Name: label, dtype: int64

### 2. 데이터 전처리

In [9]:
# 결측치 확인
df.isna().sum()

ratings    0
reviews    0
label      0
dtype: int64

In [10]:
# 중복 데이터 확인
df.reviews.nunique()

199908

In [11]:
# 중복 데이터 제거
df.drop_duplicates(subset=['reviews'], inplace=True)
df.shape

(199908, 3)

In [13]:
# 한글 이외의 데이터는 제거
df.reviews = df.reviews.str.replace('[^ㄱ-ㅎ ㅏ-ㅣ가-힣]', '')

<ipython-input-13-eb5c039868da>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.reviews = df.reviews.str.replace('[^ㄱ-ㅎ ㅏ-ㅣ가-힣]', '')


In [14]:
# 한글 이외의 데이터를 제거함으로써 발생하는 결측치가 있는지 확인
df.reviews.replace('', np.nan, inplace=True)
df.isna().sum()

ratings    0
reviews    0
label      0
dtype: int64

### 3. 토큰화

In [15]:
from konlpy.tag import Mecab
mecab = Mecab()

In [16]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [17]:
from tqdm.notebook import tqdm

data = []
for sentence in tqdm(df.reviews.values):
    morphs = mecab.morphs(sentence)
    tmp_x = [word for word in morphs if word not in stopwords]
    data.append(tmp_x)

  0%|          | 0/199908 [00:00<?, ?it/s]

### 4. 인코딩

In [20]:
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)
from tensorflow.keras.preprocessing.text import Tokenizer

In [21]:
t = Tokenizer()
t.fit_on_texts(data)

In [22]:
len(t.word_index)

45660

In [23]:
# 등장 빈도가 3 미만인 것의 갯수
threshold = 3
total_cnt = len(t.word_index)
rare_cnt = 0            # 등장 빈도가 threshold 보다 작은 단어의 갯수
total_freq = 0          # 전체 단어의 빈도수의 합
rare_freq = 0           # 등장 빈도가 threshold 보다 작은 단어의 등장 빈도수의 합

In [24]:
for key, value in t.word_counts.items():
    total_freq += value
    if value < threshold:
        rare_cnt += 1
        rare_freq += value

In [26]:
print(f'단어 집합(vocabulary)의 크기: {total_cnt}')
print(f'등장빈도가 threshold 보다 작은 단어의 갯수: {rare_cnt}')
print(f'희귀 단어의 비율: {rare_cnt / total_cnt * 100:.2f}%')
print(f'희귀 단어 등장빈도의 비율: {rare_freq / total_freq * 100:.2f}%')

단어 집합(vocabulary)의 크기: 45660
등장빈도가 threshold 보다 작은 단어의 갯수: 26311
희귀 단어의 비율: 57.62%
희귀 단어 등장빈도의 비율: 0.97%


- 등장 빈도가 threshold(3) 미만인 단어는 제외하고 토큰화

In [27]:
# 0: padding, 1: OOV(Out-of-Value)
vocab_size = total_cnt - rare_cnt + 2
vocab_size

19351

In [28]:
t = Tokenizer(num_words=vocab_size, oov_token='OOV')
t.fit_on_texts(data)
encoded_data = t.texts_to_sequences(data)

In [29]:
# 문장의 최대 길이, 평균 길이
max(len(s) for s in encoded_data), sum(map(len, encoded_data)) / len(encoded_data)

(86, 16.405471516897773)

In [30]:
# 리뷰 길이를 40으로 설정하고 패딩
max_len = 40

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_data = pad_sequences(encoded_data, maxlen=max_len)

- 훈련/테스트 데이터셋 분리

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_data, df.label.values, stratify=df.label.values, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((159926, 40), (39982, 40), (159926,), (39982,))

### 5. GRU 모델로 훈련
- 임베딩 벡터의 차원: 100
- GRU 유닛의 갯수: 128

In [33]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [34]:
model = Sequential([ 
    Embedding(vocab_size, 100, input_length=max_len),
    GRU(128),
    Dense(1, 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 100)           1935100   
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,023,549
Trainable params: 2,023,549
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path = 'best_model.h5'
mc = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
es = EarlyStopping(patience=3)

In [36]:
hist = model.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=128, callbacks=[mc, es]
)

Epoch 1/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2879 - accuracy: 0.8875
Epoch 1: val_loss improved from inf to 0.23929, saving model to best_model.h5
1000/1000 [==============================] - 204s 200ms/step - loss: 0.2879 - accuracy: 0.8875 - val_loss: 0.2393 - val_accuracy: 0.9118
Epoch 2/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2109 - accuracy: 0.9243
Epoch 2: val_loss improved from 0.23929 to 0.22583, saving model to best_model.h5
1000/1000 [==============================] - 174s 174ms/step - loss: 0.2109 - accuracy: 0.9243 - val_loss: 0.2258 - val_accuracy: 0.9169
Epoch 3/30
1000/1000 [==============================] - ETA: 0s - loss: 0.1766 - accuracy: 0.9377
Epoch 3: val_loss did not improve from 0.22583
1000/1000 [==============================] - 175s 175ms/step - loss: 0.1766 - accuracy: 0.9377 - val_loss: 0.2360 - val_accuracy: 0.9148
Epoch 4/30
1000/1000 [==============================] - ETA: 0s - loss: 0.1510 - accurac

In [37]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

1250/1250 [==============================] - 21s 16ms/step - loss: 0.2317 - accuracy: 0.9146


[0.23170846700668335, 0.9146115779876709]

### 6. 실제 데이터 예측

In [38]:
def sentiment_predict(review, tokenizer=t, max_len=max_len):
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review).strip()
    morphs = mecab.morphs(review)
    morphs = [word for word in morphs if word not in stopwords]
    encoded = tokenizer.texts_to_sequences([morphs])
    padded = pad_sequences(encoded, maxlen=max_len)
    score = float(best_model.predict(padded, verbose=0))
    return f'긍정({score*100:.2f}%)' if score > 0.5 else f'부정({(1-score)*100:.2f}%)'

In [39]:
sentiment_predict('이 상품 진짜 좋아요... 저는 강추합니다.')

'긍정(96.28%)'

In [40]:
sentiment_predict('진짜 배송도 늦고 개짜증나네요.')

'부정(99.18%)'